In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import seaborn as sns
import gc
import time
from tqdm.auto import tqdm
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn.functional as F
import random
import torch.multiprocessing

torch.multiprocessing.set_sharing_strategy('file_system')


# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

set_seed()

# Device setup for T4 GPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Constants optimized for T4 GPU (16GB VRAM)
BATCH_SIZE = 12  # Adjusted for T4 GPU
NUM_EPOCHS = 30
LEARNING_RATE = 0.0001
SEQUENCE_LENGTH = 15
FRAME_SIZE = (128, 128)
LSTM_HIDDEN_SIZE = 256
NUM_CLASSES = 2
USE_MIXED_PRECISION = True  # Enable mixed precision training for T4
NUM_WORKERS = 4  # Appropriate for 32GB system RAM

# T4 specific optimizations
torch.backends.cudnn.benchmark = True


# Paths
ROOT_DIR = "/content/drive/MyDrive/Celeb-DF"
METADATA_DIR = os.path.join(ROOT_DIR, "processed_data/metadata")
FACE_DIR = os.path.join(ROOT_DIR, "processed_data/face_clips")
OUTPUT_DIR = os.path.join(ROOT_DIR, "model_output")
CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


# Optimize memory usage
class MemoryOptimizer:
    @staticmethod
    def free_memory():
        gc.collect()
        torch.cuda.empty_cache()

    @staticmethod
    def optimize_for_inference():
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

    @staticmethod
    def optimize_for_training():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False

    @staticmethod
    def print_gpu_memory():
        if torch.cuda.is_available():
            print(f"GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f}GB allocated, "
                  f"{torch.cuda.memory_reserved() / 1024**3:.2f}GB reserved")

# Initialize memory optimizer
memory_optimizer = MemoryOptimizer()
memory_optimizer.optimize_for_training()

# Memory-optimized Dataset class
class DeepfakeDataset(Dataset):
    def __init__(self, metadata_path, face_dir, transform=None):
        self.metadata = pd.read_csv(metadata_path)
        self.face_dir = face_dir
        self.transform = transform

        # Process paths for easier loading
        self.data = []
        for _, row in self.metadata.iterrows():
            # The path in metadata contains the full path to the .npy file
            file_path = row['path']
            label = row['label']
            self.data.append((file_path, label))

        print(f"Dataset size: {len(self.data)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            path, label = self.data[idx]

            # Load face clip
            face_clip = np.load(path)

            # Ensure we have the correct number of frames
            if face_clip.shape[0] < SEQUENCE_LENGTH:
                # Duplicate last frame if needed
                last_frame = face_clip[-1]
                padding = np.array([last_frame] * (SEQUENCE_LENGTH - face_clip.shape[0]))
                face_clip = np.concatenate([face_clip, padding])
            elif face_clip.shape[0] > SEQUENCE_LENGTH:
                face_clip = face_clip[:SEQUENCE_LENGTH]

            # Convert to float32 for better compatibility
            face_clip = face_clip.astype(np.float32)

            # Apply transform if provided
            if self.transform:
                transformed_clip = torch.zeros((SEQUENCE_LENGTH, 3, FRAME_SIZE[0], FRAME_SIZE[1]),
                                              dtype=torch.float32)
                for i, frame in enumerate(face_clip):
                    transformed_clip[i] = self.transform(frame)
                face_clip = transformed_clip
            else:
                # Convert to tensor
                face_clip = torch.from_numpy(face_clip).float()
                # Swap dimensions to match expected input: [T, C, H, W]
                face_clip = face_clip.permute(0, 3, 1, 2)

            return face_clip, torch.tensor(label, dtype=torch.long)

        except Exception as e:
            print(f"Error loading {path}: {e}")
            # Return a zeros tensor with the correct shape
            return torch.zeros((SEQUENCE_LENGTH, 3, FRAME_SIZE[0], FRAME_SIZE[1]),
                              dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Optimized Model definition for T4 GPU
class ResNetLSTM(nn.Module):
    def __init__(self, lstm_hidden_size=LSTM_HIDDEN_SIZE, num_classes=NUM_CLASSES):
        super(ResNetLSTM, self).__init__()

        # Load ResNet-18 pretrained on ImageNet with memory optimization
        resnet = resnet18(weights='IMAGENET1K_V1')

        # Remove the last fully connected layer
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])

        # Feature dimension from ResNet-18
        resnet_output_dim = 512

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=resnet_output_dim,
            hidden_size=lstm_hidden_size,
            num_layers=2,
            batch_first=True,
            dropout=0.2,
            bidirectional=True
        )

        # Fully connected layer for classification
        self.fc = nn.Sequential(
            nn.Linear(lstm_hidden_size * 2, 256),  # 2 for bidirectional
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        batch_size, seq_len, c, h, w = x.size()

        # Reshape for ResNet
        x = x.view(batch_size * seq_len, c, h, w)

        # Extract features using ResNet
        features = self.backbone(x)

        # Reshape for LSTM
        features = features.view(batch_size, seq_len, -1)

        # Pass through LSTM
        lstm_out, _ = self.lstm(features)

        # Use the last time step output
        out = self.fc(lstm_out[:, -1, :])

        return out

# Training functions optimized for T4 GPU
def train_epoch(model, dataloader, criterion, optimizer, device, scaler=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    batch_times = []

    for inputs, labels in tqdm(dataloader, desc="Training"):
        batch_start = time.time()

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        if USE_MIXED_PRECISION and scaler is not None:
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        batch_end = time.time()
        batch_times.append(batch_end - batch_start)

        # Monitor GPU memory periodically
        #if random.random() < 0.1:  # Only check ~10% of batches to reduce overhead
           # memory_optimizer.print_gpu_memory

    # Calculate average batch time
    avg_batch_time = np.mean(batch_times)

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total

    print(f"Average batch time: {avg_batch_time:.4f}s")

    return epoch_loss, epoch_acc


def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)

            if USE_MIXED_PRECISION:
                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
            else:
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            running_loss += loss.item()

            probs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())  # Probability of fake class

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total

    # Calculate additional metrics
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    return epoch_loss, epoch_acc, precision, recall, f1, all_preds, all_labels, all_probs

# T4 optimized training loop
def train_model():
    print(f"Training model with the following settings:")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Device: {DEVICE}")
    print(f"Mixed precision: {USE_MIXED_PRECISION}")
    print(f"Frame size: {FRAME_SIZE}")
    print(f"Sequence length: {SEQUENCE_LENGTH}")

    # Initialize mixed precision scaler for T4
    scaler = torch.amp.GradScaler() if USE_MIXED_PRECISION else None

    # Define transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(FRAME_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create datasets
    train_dataset = DeepfakeDataset(
        os.path.join(METADATA_DIR, "train_metadata.csv"),
        FACE_DIR,
        transform=transform
    )

    val_dataset = DeepfakeDataset(
        os.path.join(METADATA_DIR, "val_metadata.csv"),
        FACE_DIR,
        transform=transform
    )

    # Create dataloaders optimized for T4
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        drop_last=True,
        persistent_workers=True  # Enable for T4 with sufficient RAM
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True  # Enable for T4 with sufficient RAM
    )

    # Initialize model
    model = ResNetLSTM().to(DEVICE)

    # Print model summary
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable parameters: {trainable_params:,}")

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3, verbose=True
    )

    # Training loop
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Val F1: {val_f1:.4f}")
        print(f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}")
        memory_optimizer.print_gpu_memory()

        # Update learning rate
        scheduler.step(val_loss)

        # Save checkpoint for every epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_f1': val_f1,
            'val_acc': val_acc,
            'val_loss': val_loss,
        }, os.path.join(CHECKPOINT_DIR, f'checkpoint_epoch_{epoch+1}.pth'))

        # Save best model
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': val_f1,
                'val_acc': val_acc,
                'val_precision': val_precision,
                'val_recall': val_recall,
            }, os.path.join(OUTPUT_DIR, 'best_model.pth'))
            print("Saved best model!")
            # Reset early stopping counter
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            print(f"Early stopping counter: {early_stopping_counter}/{early_stopping_patience}")

            if early_stopping_counter >= early_stopping_patience:
                print("Early stopping triggered!")
                break

        # Save training history
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_f1s.append(val_f1)

        # Save training history as CSV
        history_df = pd.DataFrame({
            'epoch': list(range(1, epoch+2)),
            'train_loss': train_losses,
            'val_loss': val_losses,
            'val_f1': val_f1s
        })
        history_df.to_csv(os.path.join(OUTPUT_DIR, 'training_history.csv'), index=False)

        # Save confusion matrix at the end of each epoch
        cm = confusion_matrix(val_labels, val_preds)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix - Epoch {epoch+1}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.savefig(os.path.join(OUTPUT_DIR, f'confusion_matrix_epoch_{epoch+1}.png'))
        plt.close()

        # Free up memory
        memory_optimizer.free_memory()

    # Save final model
    torch.save({
        'epoch': NUM_EPOCHS,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, os.path.join(OUTPUT_DIR, 'final_model.pth'))

    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR, 'loss_plot.png'))
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(val_f1s, label='F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.title('Validation F1 Score')
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR, 'f1_plot.png'))
    plt.close()

    return modelbest_val_f1 = 0.0
    early_stopping_counter = 0
    early_stopping_patience = 7
    train_losses = []
    val_losses = []
    val_f1s = []

    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")

        # Train
        epoch_start_time = time.time()
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, DEVICE, scaler)

        # Free some memory before validation
        memory_optimizer.free_memory()

        # Validate
        val_loss, val_acc, val_precision, val_recall, val_f1, val_preds, val_labels, val_probs = validate(
            model, val_loader, criterion, DEVICE
        )

        epoch_end_time = time.time()
        epoch_time = epoch_end_time - epoch_start_time

        # Print metrics
        print(f"Epoch completed in {epoch_time:.2f}s")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")


# T4 optimized test function
def evaluate_model(model_path):
    # Load the best model
    model = ResNetLSTM().to(DEVICE)
    checkpoint = torch.load(model_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Define transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(FRAME_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create test dataset
    test_dataset = DeepfakeDataset(
        os.path.join(METADATA_DIR, "test_metadata.csv"),
        FACE_DIR,
        transform=transform
    )

    # Create test dataloader optimized for T4
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE * 2,  # Can use larger batch for inference
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True
    )

    # Optimize for inference
    memory_optimizer.optimize_for_inference()

    # Evaluate
    criterion = nn.CrossEntropyLoss()
    test_loss, test_acc, test_precision, test_recall, test_f1, test_preds, test_labels, test_probs = validate(
        model, test_loader, criterion, DEVICE
    )

    # Print metrics
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print(f"Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}")

    # Save confusion matrix
    cm = confusion_matrix(test_labels, test_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix - Test Set')

    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(OUTPUT_DIR, 'test_confusion_matrix.png'))
    plt.close()

    # Calculate ROC curve and AUC
    try:
        roc_auc = roc_auc_score(test_labels, test_probs)
        print(f"Test ROC AUC: {roc_auc:.4f}")

        # Save ROC curve
        from sklearn.metrics import roc_curve
        fpr, tpr, _ = roc_curve(test_labels, test_probs)
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.3f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.savefig(os.path.join(OUTPUT_DIR, 'roc_curve.png'))
        plt.close()
    except Exception as e:
        print(f"Could not calculate ROC AUC: {e}")

    # Save test results
    test_results = {
        'accuracy': test_acc,
        'precision': test_precision,
        'recall': test_recall,
        'f1': test_f1,
        'confusion_matrix': cm.tolist()
    }

    pd.DataFrame([test_results]).to_csv(os.path.join(OUTPUT_DIR, 'test_results.csv'), index=False)

    # Reset optimization for training
    memory_optimizer.optimize_for_training()

    # Cleanup
    memory_optimizer.free_memory()

    return test_results

if __name__ == "__main__":
    # Print initial GPU memory usage
    memory_optimizer.print_gpu_memory()

    # Training
    print("\nStarting training...")
    model = train_model()

    # Evaluation
    print("\nEvaluating best model...")
    test_results = evaluate_model(os.path.join(OUTPUT_DIR, 'best_model.pth'))

    print("\nTraining and evaluation complete!")

    # Final test results
    print("\nFinal Test Results:")
    print(f"Accuracy: {test_results['accuracy']:.2f}%")
    print(f"Precision: {test_results['precision']:.4f}")
    print(f"Recall: {test_results['recall']:.4f}")
    print(f"F1 Score: {test_results['f1']:.4f}")

    # Free up memory
    memory_optimizer.free_memory()

Using device: cuda
GPU Memory: 0.00GB allocated, 0.00GB reserved

Starting training...
Training model with the following settings:
Batch size: 12
Device: cuda
Mixed precision: True
Frame size: (128, 128)
Sequence length: 15
Dataset size: 3046
Dataset size: 436


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 166MB/s]


Total parameters: 14,462,274
Trainable parameters: 14,462,274
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training:   0%|          | 0/253 [00:00<?, ?it/s]

...

GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15GB reserved
GPU Memory: 0.27GB allocated, 1.15

Validation:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch completed in 25.80s
Train Loss: 0.0091, Train Acc: 99.74%
Val Loss: 0.1708, Val Acc: 95.18%, Val F1: 0.9515
Val Precision: 0.9581, Val Recall: 0.9450
GPU Memory: 0.24GB allocated, 0.76GB reserved
Early stopping counter: 7/7
Early stopping triggered!

Evaluating best model...
Dataset size: 872


Validation:   0%|          | 0/37 [00:00<?, ?it/s]

Test Loss: 0.1235, Test Acc: 96.90%
Test Precision: 0.9701, Test Recall: 0.9679, Test F1: 0.9690
Test ROC AUC: 0.9939

Training and evaluation complete!

Final Test Results:
Accuracy: 96.90%
Precision: 0.9701
Recall: 0.9679
F1 Score: 0.9690
